# Benders decomposition with decision optimization

This tutorial includes everything you need to set up decision optimization engines, build a mathematical programming model, then use the benders decomposition on it.

Table of contents:

-  [Describe the business problem](#Describe-the-business-problem:--Games-Scheduling-in-the-National-Football-League)
*  [How decision optimization (prescriptive analytics) can help](#How--decision-optimization-can-help)
*  [Use decision optimization](#Use-decision-optimization)
    *  [Step 1: Set up the prescriptive model](#Step-1:-Set-up-the-prescriptive-model)
    *  [Step 2: Solve the problem with default CPLEX algorithm](#Step-2:-Solve-the-problem-with-default-CPLEX-algorithm)
    *  [Step 3: Apply a Benders strategy](#Step-3:-Apply-a-Benders-strategy)
    *  [Step 4: Use the CPLEX annotations to guide CPLEX in your Benders decomposition](#Step-4:-Use-the-CPLEX-annotations-to-guide-CPLEX-in-your-Benders-decomposition)
*  [Summary](#Summary)
****

Benders decomposition is an approach to solve mathematical programming problems with a decomposable structure.

Since V12.7, CPLEX can decompose the model into a single master and (possibly multiple) subproblems. 

To do so, CPLEX can use annotations that you supply for your model or can automatically do the decomposition. 
This approach can be applied to mixed-integer linear programs (MILP). For certain types of problems, this approach can offer significant performance improvements.

**Note**:
If your problem does not match such decomposition, CPLEX will raise an error at solve time.

CPLEX will produce an error CPXERR_BAD_DECOMPOSITION if the annotated decomposition does not yield disjoint subproblems

#### Learn more about Benders decomposition

The popular acceptance of the original paper suggesting a decomposition or partitioning of a model to support solution of mixed integer programs gave rise to "Benders algorithm" as the name.

* *J. Benders*. <i>Partitioning procedures for solving mixed-variables programming problems in Numerische Mathematik, volume 4, issue 1, pages 238–252, 1962</i>

Other researchers developed the theory of cut-generating linear programs (CGLP) to further this practice.
* *M. Fischetti, D. Salvagnin, A. Zanette*. <i>A note on the selection of Benders’ cuts in Mathematical Programming, series B, volume 124, pages 175-182, 2010</i>

Others applied the practice to practical operations research. This technical report describes Benders algorithm in "modern" terms and offers implementation hints.
* *M. Fischetti, I. Ljubic, M. Sinnl*. <i>Benders decomposition without separability: a computational study for capacitated facility location problems in Technical Report University of Padova, 2016</i>

## How  decision optimization can help

* Prescriptive analytics (decision optimization) technology recommends actions that are based on desired outcomes.  It takes into account specific scenarios, resources, and knowledge of past and current events. With this insight, your organization can make better decisions and have greater control of business outcomes.  

* Prescriptive analytics is the next step on the path to insight-based actions. It creates value through synergy with predictive analytics, which analyzes data to predict future outcomes.  

* Prescriptive analytics takes that insight to the next level by suggesting the optimal way to handle that future situation. Organizations that can act fast in dynamic conditions and make superior decisions in uncertain environments gain a strong competitive advantage.  
<br/>

<u>With prescriptive analytics, you can:</u> 

* Automate the complex decisions and trade-offs to better manage your limited resources.
* Take advantage of a future opportunity or mitigate a future risk.
* Proactively update recommendations based on changing events.
* Meet operational goals, increase customer loyalty, prevent threats and fraud, and optimize business processes.



## Use decision optimization

### Step 1: Set up the prescriptive model

Write a toy model to show how to use the annotation api.

This model is not important: it just matches a Benders decomposition, that is CPLEX can apply its new algorithm without any error.

The aim of this notebook is to discover and learn how to successfully apply Benders decomposition, not to see huge performance differences between a standard solve and a Benders-based solve.

In [1]:
d1 = 25
d2 = 35

Costs = [[20,  15,  22,  27,  13,  4,  15,  6,  15,  22,  25,  13,  7,  28,  14,  5,  8,  1,  17,  3,  19,  17,  22,  12,  14],
   [2,  15,  16,  16,  10,  13,  4,  2,  6,  29,  10,  8,  20,  11,  8,  11,  28,  17,  10,  29,  3,  24,  12,  11,  11],
   [13,  14,  6,  17,  14,  13,  8,  29,  19,  26,  22,  0,  8,  29,  15,  20,  5,  20,  26,  17, 24, 10, 24, 9, 1],
   [7, 27, 24, 3, 4, 23, 11, 9, 18, 1, 29, 24, 16, 9, 8, 3, 18, 24, 10, 12, 1, 3, 15, 29, 3],
   [25, 26, 29, 6, 24, 8, 2, 10, 17, 0, 4, 7, 2, 17, 2, 27, 24, 20, 18, 5, 5, 2, 21, 26, 20],
   [29, 5, 15, 5, 4, 26, 18, 8, 2, 14, 13, 6, 14, 28, 16, 28, 23, 8, 5, 8, 10, 28, 17, 0, 23],
   [12, 16, 10, 16, 17, 10, 29, 11, 28, 22, 25, 8, 27, 12, 10, 28, 7, 5, 3, 9, 18, 10, 15, 16, 2],
   [12, 9, 14, 23, 26, 4, 3, 3, 22, 12, 11, 9, 19, 5, 6, 16, 1, 1, 9, 20, 23, 23, 27, 4, 11],
   [18, 13, 28, 29, 3, 28, 16, 11, 9, 2, 7, 20, 13, 23, 6, 10, 3, 16, 14, 2, 15, 17, 1, 19, 27],
   [29, 17, 17, 14, 21, 18, 8, 21, 9, 20, 14, 6, 29, 24, 24, 4, 18, 16, 21, 24, 26, 0, 26, 9, 5],
   [27, 24, 21, 28, 17, 18, 10, 10, 26, 25, 13, 18, 2, 9, 16, 26, 10, 22, 5, 17, 15, 0, 9, 0, 16],
   [13, 15, 17, 21, 25, 9, 22, 13, 20, 15, 1, 17, 18, 10, 2, 27, 19, 21, 14, 26, 29, 13, 28, 28, 15],
   [16, 12, 2, 2, 9, 27, 11, 14, 12, 2, 14, 29, 3, 12, 18, 6, 7, 9, 1, 5, 19, 14, 11, 29, 4],
   [1, 15, 27, 29, 16, 17, 10, 10, 17, 19, 6, 10, 20, 20, 19, 10, 19, 26, 15, 7, 20, 19, 13, 3, 22],
   [22, 14, 12, 3, 22, 6, 15, 3, 6, 10, 9, 13, 11, 21, 6, 19, 29, 4, 5, 21, 7, 12, 13, 11, 22],
   [9, 27, 22, 29, 11, 14, 1, 19, 21, 2, 4, 13, 17, 9, 10, 17, 13, 8, 24, 13, 26, 27, 23, 4, 21],
   [3, 14, 26, 18, 17, 3, 1, 11, 13, 8, 22, 3, 18, 26, 17, 15, 22, 10, 19, 23, 13, 14, 17, 18, 27],
   [21, 14, 1, 28, 28, 0, 0, 29, 12, 23, 22, 17, 19, 2, 10, 19, 4, 18, 28, 13, 27, 12, 9, 29, 22],
   [29, 3, 20, 5, 5, 23, 28, 16, 1, 8, 26, 23, 11, 11, 21, 17, 13, 21, 3, 8, 6, 15, 18, 6, 24],
   [14, 20, 26, 10, 17, 20, 5, 9, 25, 20, 14, 22, 5, 12, 0, 18, 7, 0, 8, 15, 21, 12, 26, 7, 21],
   [7, 7, 1, 9, 24, 29, 0, 3, 29, 24, 1, 6, 14, 0, 11, 5, 21, 12, 15, 1, 25, 4, 7, 17, 16],
   [8, 18, 15, 6, 1, 22, 26, 13, 19, 20, 12, 15, 19, 27, 13, 3, 22, 22, 22, 20, 0, 4, 24, 13, 25],
   [14, 6, 29, 23, 8, 5, 4, 18, 21, 29, 18, 2, 2, 3, 7, 13, 12, 9, 2, 18, 26, 3, 18, 7, 7],
   [5, 8, 4, 8, 25, 4, 6, 20, 14, 21, 18, 16, 15, 11, 7, 8, 20, 27, 22, 7, 5, 8, 24, 11, 8],
   [0, 8, 29, 25, 29, 0, 12, 25, 19, 9, 19, 25, 27, 21, 2, 23, 2, 25, 17, 6, 0, 6, 15, 2, 15],
   [23, 24, 10, 26, 7, 5, 5, 26, 1, 16, 22, 8, 24, 9, 16, 17, 1, 26, 20, 23, 18, 20, 23, 2, 19],
   [16, 3, 9, 21, 15, 29, 8, 26, 20, 12, 18, 27, 29, 15, 24, 9, 17, 24, 3, 5, 21, 28, 7, 1, 12],
   [1, 11, 21, 1, 13, 14, 16, 14, 17, 25, 18, 9, 19, 26, 1, 13, 15, 6, 14, 10, 12, 19, 0, 15, 7],
   [20, 14, 7, 5, 8, 16, 12, 0, 5, 14, 18, 16, 24, 27, 20, 7, 11, 3, 16, 8, 2, 2, 4, 0, 3],
   [26, 19, 27, 29, 8, 9, 8, 10, 18, 4, 6, 0, 5, 17, 12, 18, 17, 17, 13, 0, 16, 12, 18, 19, 16],
   [3, 12, 11, 28, 3, 2, 14, 14, 17, 29, 18, 14, 19, 24, 9, 27, 4, 19, 6, 24, 19, 3, 28, 20, 4],
   [2, 0, 21, 14, 21, 12, 27, 6, 20, 29, 13, 21, 23, 0, 20, 4, 11, 27, 3, 11, 21, 11, 21, 4, 17],
   [20, 26, 5, 8, 18, 14, 12, 12, 24, 3, 8, 0, 25, 16, 19, 21, 7, 4, 23, 21, 20, 28, 6, 21, 19],
   [16, 18, 9, 1, 9, 7, 14, 6, 28, 26, 3, 14, 27, 4, 9, 9, 1, 9, 24, 3, 14, 13, 18, 3, 27],
   [1, 19, 7, 20, 26, 27, 0, 7, 4, 0, 13, 8, 10, 17, 14, 19, 21, 21, 14, 15, 22, 14, 5, 27, 0]];

In [2]:
R1 = range(1,d1)
R2 = range(1,d2);

dim  = range(1,d1*d2+1)

Create one model instance, with a name. Set the log output to true so that you can see when CPLEX enables the Benders algorithm.

In [3]:
# first import the Model class from docplex.mp
from docplex.mp.model import Model

m = Model(name='benders', log_output=True)

In [4]:
X = m.continuous_var_dict([(i,j) for i in R2 for j in R1])
Y = m.integer_var_dict(R1, 0, 1)


bendersPartition = {(i,j) : i for i in R2 for j in R1}

In [5]:
m.minimize( m.sum( Costs[i][j]*X[i,j] for i in R2 for j in R1) + sum(Y[i] for i in R1) )


m.add_constraints( m.sum( X[i,j] for j in R1) ==1 for i in R2)
    
m.add_constraints( X[i,j] - Y[j] <= 0 for i in R2 for j in R1)

[docplex.mp.LinearConstraint[](_x1-_x817,LE,0),
 docplex.mp.LinearConstraint[](_x2-_x818,LE,0),
 docplex.mp.LinearConstraint[](_x3-_x819,LE,0),
 docplex.mp.LinearConstraint[](_x4-_x820,LE,0),
 docplex.mp.LinearConstraint[](_x5-_x821,LE,0),
 docplex.mp.LinearConstraint[](_x6-_x822,LE,0),
 docplex.mp.LinearConstraint[](_x7-_x823,LE,0),
 docplex.mp.LinearConstraint[](_x8-_x824,LE,0),
 docplex.mp.LinearConstraint[](_x9-_x825,LE,0),
 docplex.mp.LinearConstraint[](_x10-_x826,LE,0),
 docplex.mp.LinearConstraint[](_x11-_x827,LE,0),
 docplex.mp.LinearConstraint[](_x12-_x828,LE,0),
 docplex.mp.LinearConstraint[](_x13-_x829,LE,0),
 docplex.mp.LinearConstraint[](_x14-_x830,LE,0),
 docplex.mp.LinearConstraint[](_x15-_x831,LE,0),
 docplex.mp.LinearConstraint[](_x16-_x832,LE,0),
 docplex.mp.LinearConstraint[](_x17-_x833,LE,0),
 docplex.mp.LinearConstraint[](_x18-_x834,LE,0),
 docplex.mp.LinearConstraint[](_x19-_x835,LE,0),
 docplex.mp.LinearConstraint[](_x20-_x836,LE,0),
 docplex.mp.LinearConstraint[

#### Solve with Decision Optimization 

Depending on the size of the problem, the solve stage might fail and require the Commercial Edition of CPLEX engines, which is included in the premium environments in Watson Studio.

You will get the best solution found after ***n*** seconds, because of a time limit parameter.

In [6]:
m.print_information()

Model: benders
 - number of variables: 840
   - binary=0, integer=24, continuous=816
 - number of constraints: 850
   - linear=850
 - parameters: defaults


### Step 2: Solve the problem with default CPLEX algorithm

In [7]:
msol = m.solve()
assert msol is not None, "model can't solve"
m.report()

CPXPARAM_Read_DataCheck                          1
Found incumbent of value 494.000000 after 0.00 sec. (0.07 ticks)
Tried aggregator 1 time.
Reduced MIP has 850 rows, 840 columns, and 2448 nonzeros.
Reduced MIP has 24 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.09 ticks)
Probing time = 0.00 sec. (0.10 ticks)
Tried aggregator 1 time.
Reduced MIP has 850 rows, 840 columns, and 2448 nonzeros.
Reduced MIP has 24 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.36 ticks)
Probing time = 0.00 sec. (0.10 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 32 threads.
Root relaxation solution time = 0.00 sec. (0.59 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                          494.0000        0.0000           100.00%
*     0+    0  

#### Inspect the CPLEX Log.

If you inspect the CPLEX, you will see that it is a very standard log.
CPLEX needed 63 iterations to solve it.

In [8]:
obj1 = m.objective_value

### Step 3: Apply a Benders strategy

CPLEX implements a default Benders decomposition in certain situations.

If you want CPLEX to apply a Benders strategy as it solves your problem, but you do not specify cpxBendersPartition annotations yourself, CPLEX puts all integer variables in master and continuous variables into subproblems. 
If there are no integer variables in your model, or if there are no continuous variables in your model, CPLEX raises an error stating that it cannot automatically decompose the model to apply a Benders strategy.

You just need to set the Benders strategy parameter.

CPLEX supports 4 values for this parameter, from -1 to 3:
* OFF (default value) will ignore Benders.
* AUTO, USER, WORKERS, FULL will enable Benders.

Refer to the CPLEX documentation to understand the differences between the 4 values that trigger it.

In [9]:
m.parameters.benders.strategy = 3

In [10]:
m.print_information()

Model: benders
 - number of variables: 840
   - binary=0, integer=24, continuous=816
 - number of constraints: 850
   - linear=850
 - parameters:
     parameters.benders.strategy = 3


Call cplex solve, but with the <i>clean_before_solve</i> flag to ignore everything about the previous solve and solution.

In [11]:
msol = m.solve(clean_before_solve=True)
assert msol is not None, "model can't solve"
m.report()

CPXPARAM_Read_DataCheck                          1
CPXPARAM_Benders_Strategy                        3
Found incumbent of value 494.000000 after 0.00 sec. (0.07 ticks)
Tried aggregator 1 time.
Reduced MIP has 850 rows, 840 columns, and 2448 nonzeros.
Reduced MIP has 24 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.77 ticks)
Tried aggregator 1 time.
Reduced MIP has 850 rows, 840 columns, and 2448 nonzeros.
Reduced MIP has 24 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.72 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

      0     0        0.0000                        Benders: 2        0         
      0     0        4.0000                        Benders: 7        2         
      0     0       12.8125                        Benders: 3       11         
      0     0       15.8333                       Benders: 13       15    

#### Inspect the CPLEX Log.

Inspect the CPLEX log: you can now see that the logs are different and you can see the message
<code>
Benders cuts applied:  3
</code>
which proves CPLEX applied successfully

You can see that CPLEX needed only 61 cumulative iterations, while it needed 63 previously.

In [12]:
obj2 = m.objective_value

### Step 4: Use the CPLEX annotations to guide CPLEX in your Benders decomposition

In [13]:
m.parameters.benders.strategy = 1

Settings Benders annotation in docplex is very simple.
You just need to use the <i>benders_annotation</i> property available on variables and constraints to state which worker they belong to.

In [14]:
for i in R2:
    for j in R1:
        X[i,j].benders_annotation =  i%2

In [15]:
m.print_information()

Model: benders
 - number of variables: 840
   - binary=0, integer=24, continuous=816
 - number of constraints: 850
   - linear=850
 - annotations: 816
   - variables: 816
 - parameters:
     parameters.benders.strategy = 1


In [16]:
msol = m.solve(clean_before_solve=True)
assert msol is not None, "model can't solve"
m.report()

CPXPARAM_Read_DataCheck                          1
CPXPARAM_Benders_Strategy                        1
Found incumbent of value 494.000000 after 0.00 sec. (0.07 ticks)
Tried aggregator 1 time.
Reduced MIP has 850 rows, 840 columns, and 2448 nonzeros.
Reduced MIP has 24 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.77 ticks)
Tried aggregator 1 time.
Reduced MIP has 850 rows, 840 columns, and 2448 nonzeros.
Reduced MIP has 24 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.72 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

      0     0       24.0000                        Benders: 1       21         
      0     0       26.4615                        Benders: 1       25         
      0     0       30.7500                        Benders: 1       30         
      0     0       37.2500                        Benders: 1       34    

#### Inspect the CPLEX Log.

Inspect the CPLEX log: you can see that you now need only 57 cumulative iterations instead of 61 with default Benders and 63 with no Benders.
If you look at the <i>Best Bound</i> column, you will also see that the listed sub problems are not the same as CPLEX applied the decomposition provided by the annotations.

In [17]:
obj3 = m.objective_value

In [18]:
assert (obj1 == obj2) and (obj2 == obj3)

## Summary


You have learned how to set up and use the IBM Decision Optimization CPLEX Modeling for Python to formulate a Mathematical Programming model and apply a Benders decomposition.

#### References
* <a href="https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html" target="_blank" rel="noopener noreferrer">Decision Optimization CPLEX Modeling for Python documentation</a>
* <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html" target="_blank" rel="noopener noreferrer">Watson Studio documentation</a>


<hr>
Copyright © 2017-2019. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>